In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import glob

In [3]:
positions = ["PosX", "PosY", "PosZ"]
rotations = ["RotX", "RotY", "RotZ"]
tmp = positions + rotations


path = r'../from_study/CSV' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, delimiter=";")
    li.append(df)


def calc_euc(df):
    for pos in positions:
        df[pos + "_diff"] = df[pos] - df[pos].shift(1)
        df[pos + "_square"] = df[pos + "_diff"] * df[pos + "_diff"]

    df["euc"] = (df["PosX_square"] + df["PosY_square"] + df["PosZ_square"]).apply(lambda x: np.sqrt(x))
    return df["euc"].sum()


for j in range(len(li)):
    df = li[j]
    df = df[(df.isStudy == True)]
    print(j)
    for i in range(1, 4):
        for hand in ["Left", "Right"]:
            tmp = df[(df.Hand == hand) & (df.Condition == i)]
            print("condition", str(i), "-", hand, ":\t", calc_euc(tmp))

0


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


condition 1 - Left :	 42.75163723823512
condition 1 - Right :	 56.09072708558922
condition 2 - Left :	 32.664115276395854
condition 2 - Right :	 32.92519724101026
condition 3 - Left :	 1476.676000065342
condition 3 - Right :	 34.0716203998028
1
condition 1 - Left :	 28.103057134284626
condition 1 - Right :	 27.344571776965452
condition 2 - Left :	 24.262956270400583
condition 2 - Right :	 22.291343083715006
condition 3 - Left :	 29.377651160281008
condition 3 - Right :	 39.08680110430237
2
condition 1 - Left :	 40.51134702123985
condition 1 - Right :	 39.13231089638289
condition 2 - Left :	 22.671918497283734
condition 2 - Right :	 14.54459824361512
condition 3 - Left :	 26.48661588498237
condition 3 - Right :	 29.697913707260593
3
condition 1 - Left :	 18.201741260710737
condition 1 - Right :	 18.860004895572107
condition 2 - Left :	 10.600443289233075
condition 2 - Right :	 10.875074212666355
condition 3 - Left :	 25.124397112499825
condition 3 - Right :	 25.260734495264618
4
conditi